# やること

## パラメータは？

パラメータとして与えるのは以下

①　$\rho_{\chi_{k}}$　:　 $\chi_{k}$に対するbeliefのprior precision。　candidateごとに用意する。

②　$\mu_{\chi_{k}}$　：　$\chi_{k}$に対するbeliefのmean。candidateごとに用意する。

③　$\rho_{\eta}$　：　シグナルのprecision。一つで良い。

④　$\chi_{k}$　：　candidate kのtrue quality。candidateごとに用意する。

## belief updating

③と④をgivenとして$s_{1,k}\sim N(\chi_{k}, \frac{1}{\rho_{\eta}})$をdrawする。

1期のシグナルをもとに、①②③を用いてposteriorを次のように構成する。

$B_{1, k} \sim N\left(\frac{\rho_{\chi_{k}} \mu_{\chi_{k}} + \rho_{\eta}s_{1, k}}{\rho_{\chi_{k}} + t \rho_{\eta}}, \frac{1}{\rho_{\chi_{k}} + t \rho_{\eta}}\right)$

このposterior meanを$E\left[\chi_{k}|\Omega_{1}\right]$として記録する。

以降t期においても同様に

③と④をgivenとして$s_{t,k}\sim N(\chi_{k}, \frac{1}{\rho_{\eta}})$をdrawする。

t期のシグナルをもとに、①②③を用いてposteriorを次のように構成する。

$B_{1, k} \sim N\left(\frac{\rho_{\chi_{k}} \mu_{\chi_{k}} + \rho_{\eta} \sum_{\tau = 1}^{t} s_{\tau, k}}{\rho_{\chi_{k}} + t \rho_{\eta}}, \frac{1}{\rho_{\chi_{k}} + t \rho_{\eta}}\right)$

そしてそのposterior meanを$E\left[\chi_{k}|\Omega_{t}\right]$として記録する。

## 詳細

パラメータセット一つにつきT回シグナルを乱数発生する。使うパラメータはいつも一緒なので出てくる数字に大差はない。

違うのはその出てくる順番。T!順あり得るので一つのパラメータセットにつきT!回1期からT期までの$E\left[\chi_{k}|\Omega_{t}\right]$を記録する。

さらにパラメータは100セット用意。

以下では、candidate4人について$E\left[\chi_{k}|\Omega_{t}\right]$のpathを100×T!通り作成するためのclassを作成する。

In [1]:
import numpy as np
from scipy.special import factorial

In [74]:
class ExpectedChi:
    
    def __init__(self, lam, T, sim):
        self.lam = lam # parameter of exponential distribution which rho's follow
        self.T = T # time
        self.sim = sim # simulation number
    
    def setparam(self):
        # params[0, :]　：　rho_eta
        # params[(1,2,3,4), :] : rho_chi for each candidate
        # params[(5,6,7,8), :] : mu_chi for each candidate
        # params[(9,10,11,12), :] : chi for each candidate
        rho = np.random.exponential(self.lam, (5, self.sim))
        mu_chi = np.random.normal(0.0,1.0,(8, self.sim))
        params = np.vstack((rho, mu_chi))
        return params
        
    def signals(self):
        # make the 4 dimentional array : all_signals
        # each element in all_signals is 3 dimentional array whose shape is (T! * 4 * T )
        params = self.setparam()
        all_signals = np.empty([self.sim, factorial(self.T).astype(int), 4, self.T])
        for i in range(self.sim):
            signals_differentparam = np.empty([factorial(self.T).astype(int), 4, self.T])
            all_signals[i, :, :, :] = signals_differentparam
            for k in range(factorial(self.T).astype(int)):
                signals = np.empty([4, self.T])
                signals_differentparam[k, :, :] = signals
                for j in range(4):
                    signal = np.random.normal(params[j + 9, i], 1/params[0, i],  (1, self.T))
                    signals[j, :] = signal
        return all_signals
    
    def cum_signals(self):
        # calculating cumulative signals for expected chi_k given omega
        sig = self.signals()
        for i in range(self.sim):
            for j in range(self.T):
                if j != 0:
                    sig[i, :, :, :][:, :, j] = sig[i, :, :, :][:, :, j] + sig[i, :, :, :][:, :, j -1]
        return sig
            
    def expected_chi(self):
        # calculating expected chi_k given omega
        cum = self.cum_signals()
        params = self.setparam()
        transform = np.empty((self.sim, factorial(self.T).astype(int), 4, self.T))
        
        # first multiply cum by rho_eta 
        for i in range(self.sim):
            transform[i, :, :, :] = cum[i, :, :, :] * params[0, i]
            
        # second add rho_chi + mu_chi to transform
        for i in range(self.sim):
            for j in range(4):
                transform[i, :, j, :] = transform[i, :, j, :] + (params[j + 1, i] * params[j + 5, i])
        
        # third divide by rho_chi + t * rho_eta
        for i in range(self.sim):
            for j in range(4):
                for t in range(self.T):
                    transform[i, :, j, t] = transform[i, :, j, t] / (params[j + 1, i] + t * params[0, i])
        
        return transform

In [75]:
test = ExpectedChi(lam = 1.0, T = 3, sim = 10)

In [76]:
test.setparam()

array([[  1.42424657e-01,   2.16792313e-01,   3.24512583e-01,
          1.17230627e+00,   4.73959794e-01,   8.72099836e-01,
          1.24907481e-01,   2.68284872e-01,   4.04298661e-01,
          1.31519523e+00],
       [  1.08316337e+00,   7.43098478e-01,   1.72558379e-01,
          1.57066472e+00,   3.32097871e-01,   2.94270368e+00,
          1.78531557e-01,   8.82541998e-02,   8.78094225e-01,
          1.69854646e+00],
       [  2.61849017e+00,   7.75109510e-01,   3.62176513e+00,
          7.48824311e-01,   1.58121129e+00,   7.76084012e-02,
          1.26095970e+00,   1.27115996e-02,   1.25388990e+00,
          4.81254390e-01],
       [  1.31107529e+00,   1.77123677e-01,   9.80766525e-01,
          6.98881161e-01,   5.17140233e-01,   9.90568561e-01,
          5.99405163e-01,   6.17298638e-02,   2.06426496e-01,
          2.24774847e+00],
       [  2.37814569e+00,   2.85991278e-01,   3.73975057e+00,
          6.73636764e+00,   3.06705438e+00,   3.67137610e-01,
          2.56015236e+00

In [77]:
test.signals()

array([[[[  0.00000000e+000,   0.00000000e+000,   2.12215770e-314],
         [  1.48219694e-323,   0.00000000e+000,   0.00000000e+000],
         [  0.00000000e+000,   1.43320476e+161,   0.00000000e+000],
         [  0.00000000e+000,  -3.23551621e+024,   2.14094118e-314]],

        [[  2.12306291e-314,   3.26499285e-285,   2.14093530e-314],
         [  2.12311406e-314,   6.23606390e-042,   0.00000000e+000],
         [  0.00000000e+000,   4.76484771e+180,   0.00000000e+000],
         [  0.00000000e+000,   3.98884948e+083,   2.14094115e-314]],

        [[  2.12306291e-314,   4.66108666e+009,   2.14094122e-314],
         [  2.12306293e-314,  -8.57260926e+285,   2.13901527e-314],
         [  2.15278617e-314,   1.54535562e-146,   2.14094128e-314],
         [  2.15220943e-314,   0.00000000e+000,   0.00000000e+000]],

        [[  0.00000000e+000,   2.18229722e-251,   2.14094125e-314],
         [  2.12306291e-314,   3.71322047e+147,   2.14094131e-314],
         [  2.12306291e-314,   1.68056671e

In [78]:
test.cum_signals()

array([[[[ -5.73786322e-01,  -1.06616884e+00,  -1.81461703e+00],
         [  1.53225926e+00,   3.21814392e+00,   4.63538514e+00],
         [  3.08830041e-01,   9.71313024e-01,   1.24548308e+00],
         [  3.61120158e-01,   8.38852016e-01,   8.79590968e-01]],

        [[ -2.43499563e-01,  -5.75639914e-01,  -1.35214364e+00],
         [  1.55008329e+00,   3.13683867e+00,   4.62000578e+00],
         [ -2.99871669e-02,   5.44521385e-01,   5.83053617e-01],
         [  5.58982309e-01,   1.09817646e+00,   1.24660111e+00]],

        [[  4.33147278e+00,   8.36935469e+00,   1.05233200e+01],
         [  4.45778887e+00,   6.19278061e+00,   9.48730224e+00],
         [  1.62995587e+00,   1.00762290e+01,   2.21217217e+01],
         [ -8.90474940e+00,  -9.91344478e+00,  -5.26269812e+00]],

        [[  3.43654685e+00,   6.76224799e-01,  -2.80192588e-01],
         [ -5.93553260e+00,  -8.77702036e+00,  -4.04583570e+00],
         [ -4.80228096e-01,  -4.20723513e+00,  -1.49927402e+00],
         [ -3.08828

In [79]:
test.expected_chi()

array([[[[ -5.60035160e+00,  -3.53114952e+00,  -2.16000867e+00],
         [ -5.66053526e+00,  -3.23295439e+00,  -3.07416360e+00],
         [ -1.17450861e+00,   4.97556165e-01,   2.14087946e+00],
         [ -1.71106742e+00,  -2.94066387e+00,  -1.61688796e+00]],

        [[ -2.79112821e+00,  -4.92757119e-01,  -1.84521184e-01],
         [ -1.78852388e+00,   1.62937998e+00,   3.14916677e+00],
         [ -2.64415479e+00,   2.21491753e+00,   2.77993598e+00],
         [ -1.72846746e+00,  -1.67699571e+00,  -2.14976524e+00]],

        [[ -4.94351995e+00,  -4.64735363e+00,  -3.86376202e+00],
         [  6.01276207e-01,  -1.26965061e+00,   1.33853694e-01],
         [  8.05172147e+00,   1.91353825e+00,   7.62892632e-01],
         [ -1.99106201e+00,  -1.29823499e+00,  -7.94428892e-01]],

        [[ -3.09580237e+00,  -1.92959580e+00,  -1.20889942e+00],
         [  3.63014090e-01,   5.80223874e-01,  -4.55515104e-02],
         [ -6.96489908e-01,  -2.05161837e+00,  -2.96101079e+00],
         [ -3.68166